In [1]:
import hail as hl
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

hl.init(app_name="only_TOB_pca")

Loading BokehJS ...

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eriurn/submit/python37_intel/lib/python3.7/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.0.0-78/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-02-07 03:41:58 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-02-07 03:42:00 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-07 03:42:00 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2022-02-07 03:42:01 WARN  Client:69 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Running on Apache Spark version 3.1.2
SparkUI available at http://dice01-mlx.mlx:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.68-13190f0b6103
LOGGING: writing to /home/eriurn/Hail-joint-caller/notebooks/hail-20220207-0341-0.2.68-13190f0b6103.log


In [2]:
# Drop number of samples if hitting a memory issue
path_to_mt = "file:///home/eriurn/Hail-joint-caller/scripts/result.mt"
TOB_mt = hl.read_matrix_table(path_to_mt)

In [3]:
# Import super_population annotation for TOB
sa_TOB = hl.import_table("file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/TOB_annotations.txt", key='s')

2022-02-07 03:42:27 Hail: INFO: Reading table without type imputation
  Loading field 's' as type str (not specified)
  Loading field 'population' as type str (not specified)
  Loading field 'super_population' as type str (not specified)


In [4]:
TOB_mt = TOB_mt.annotate_cols(pheno = sa_TOB[TOB_mt.s])

In [5]:
# Convert LGT to GT [MIGHT NOT HAVE TO DO THIS ANYMORE SINCE WE'RE USING TOB_mt.LGT BELOW]
GT = hl.experimental.lgt_to_gt(TOB_mt.LGT, TOB_mt.LA)

In [8]:
# Calculate eigenvalues and pca_scores
pca_eigenvalues_TOB, pca_scores_TOB, pca_loadings_TOB = hl.hwe_normalized_pca(GT, compute_loadings=True, k=2)

2022-02-07 06:12:22 Hail: INFO: hwe_normalized_pca: running PCA using 21567582 variants.
2022-02-07 06:13:00 Hail: INFO: pca: running PCA with 2 components... 1) / 2586]
Exception in thread "task-result-getter-2" Exception in thread "task-result-getter-0" java.lang.OutOfMemoryError: GC overhead limit exceeded
	at scala.Predef$.genericArrayOps(Predef.scala:422)
	at com.twitter.chill.WrappedArraySerializer.read(WrappedArraySerializer.scala:38)
	at com.twitter.chill.WrappedArraySerializer.read(WrappedArraySerializer.scala:23)
	at com.esotericsoftware.kryo.Kryo.readObject(Kryo.java:731)
	at com.esotericsoftware.kryo.serializers.DefaultArraySerializers$ObjectArraySerializer.read(DefaultArraySerializers.java:391)
	at com.esotericsoftware.kryo.serializers.DefaultArraySerializers$ObjectArraySerializer.read(DefaultArraySerializers.java:302)
	at com.esotericsoftware.kryo.Kryo.readObject(Kryo.java:731)
	at com.esotericsoftware.kryo.serializers.ObjectField.read(ObjectField.java:125)
	at com.esoter

FatalError: OutOfMemoryError: GC overhead limit exceeded

Java stack trace:
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at 



Hail version: 0.2.68-13190f0b6103
Error summary: OutOfMemoryError: GC overhead limit exceeded

In [ ]:
# Save pca__scores_TOB
pca_scores_TOB.write('file:///home/eriurn/pca_scores_TOB.ht', overwrite=True)

In [ ]:
# Annotate the pca_scores
TOB_ht = pca_scores_TOB.select(PC1=pca_scores_TOB.scores[0], PC2=pca_scores_TOB.scores[1])
TOB_ht = TOB_ht.annotate(pheno = sa_TOB[TOB_ht.s])

In [ ]:
TOB_ht.describe()

In [ ]:
TOB_ht.pheno.show()

In [ ]:
# Currently label for TOB doesn't work
p1 = hl.plot.scatter(TOB_ht.PC1, TOB_ht.PC2, xlabel='PC1', ylabel='PC2', size=10)
#p1 = hl.plot.scatter(ht.PC1, ht.PC2, xlabel='PC1', ylabel='PC2', label=ht.pheno.super_population, size=6)
show(p1)

Exception in thread "task-result-getter-13" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-14" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-11" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-10" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-16" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "dispatcher-CoarseGrainedScheduler" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-18" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-15" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "dispatcher-CoarseGrainedScheduler" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "task-result-getter-17" java.lang.OutOfMemoryError: GC overhead l

In [ ]:
################### 1000 genomes PCA ###################
